In [1]:
%pip install pandas
%pip install opencv-python


  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.2/11.6 MB 1.5 MB/s eta 0:00:08
   - -------------------------------------- 0.5/11.6 MB 3.3 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/11.6 MB 3.5 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.6 MB 5.4 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/11.6 MB 6.5 MB/s eta 0:00:02
   ------ --------------------------------- 1.9/11.6 MB 6.2 MB/s eta 0:00:02
   -------- ------------------------------- 2.6/11.6 MB 7.2 MB/s eta 0:00:02
   ---------- ----------------------------- 3.1/11.6 MB 7.7 MB/s eta 0:00:02
   ------------ -----------------------

In [2]:
import pandas as pd
import cv2
import os

In [3]:
header_names = ["image_name", "face_id_in_image", "face_box_top", "face_box_left", "face_box_right", "face_box_bottom" ,"face_box_confidence", "expression_label"]
label_df = pd.read_csv("../../data/raw/expw/label/label.csv",names=header_names,  delimiter=" ", index_col=False)

label_df.head()

,image_name,face_id_in_image,face_box_top,face_box_left,face_box_right,face_box_bottom,face_box_confidence,expression_label
0,angry_actor_104.jpg,0,28,113,226,141,22.9362,0
1,angry_actor_109.jpg,0,31,157,345,219,50.3056,0
2,angry_actor_120.jpg,1,53,53,372,372,13.9434,2
3,angry_actor_13.jpg,0,77,51,362,388,85.8104,3
4,angry_actor_132.jpg,0,95,31,412,476,82.3948,0


In [4]:
expression_labels = {
    0: "angry",
    1: "disgust",
    2: "fear",
    3: "happy",
    4: "sad",
    5: "surprise",
    6: "neutral",
}

In [5]:
for index, row in label_df.iterrows():
    label_df.loc[index, "expression_label"] = expression_labels.get(row["expression_label"])

label_df["expression_label"] = label_df["expression_label"].astype("category")

label_df["expression_label"].head()

C:\Users\pedro\AppData\Local\Temp\ipykernel_16672\3422431066.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'angry' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  label_df.loc[index, "expression_label"] = expression_labels.get(row["expression_label"])


0    angry
1    angry
2     fear
3    happy
4    angry
Name: expression_label, dtype: category
Categories (7, object): ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [6]:
img_folder_path = "../../data/raw/expw/img/"
margin = 0.1

In [7]:
images = []
image_names = []
for index, row in label_df.iterrows():
    img = cv2.imread(img_folder_path + row["image_name"])

    facebox_right = row["face_box_right"]
    facebox_left = row["face_box_left"]
    facebox_bottom = row["face_box_bottom"]
    facebox_top = row["face_box_top"]

    width_margin = int((facebox_right - facebox_left) * margin)
    height_margin = int((facebox_bottom - facebox_top) * margin)

    adjusted_top = max(facebox_top - height_margin, 0)
    adjusted_bottom = min(facebox_bottom + height_margin, img.shape[0])
    adjusted_left = max(facebox_left - width_margin, 0)
    adjusted_right = min(facebox_right + width_margin, img.shape[1])


    image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    cropped_img_with_margin = image_gray[
        adjusted_top:adjusted_bottom, adjusted_left:adjusted_right
    ]
    resized_img = cv2.resize(cropped_img_with_margin, (48, 48))
    images.append(resized_img)
    image_names.append(
        row["expression_label"] + "_" + str(index) + ".jpg"
    )

In [8]:
processed_folder_path = "../../data/processed/expw/cropped/"

if not os.path.exists(processed_folder_path):
    os.makedirs(processed_folder_path)

for i, img in enumerate(images):
    cv2.imwrite(
        os.path.join(
            processed_folder_path, image_names[i]
        ),
        img,
    )

In [9]:
new_label_df = label_df
new_label_df["image_name"] = image_names
new_label_df.drop(columns=["face_id_in_image", "face_box_top", "face_box_left", "face_box_right", "face_box_bottom", "face_box_confidence"], inplace=True)

new_label_df.head()

,image_name,expression_label
0,angry_0.jpg,angry
1,angry_1.jpg,angry
2,fear_2.jpg,fear
3,happy_3.jpg,happy
4,angry_4.jpg,angry


In [10]:
label_path = "../../data/processed/expw/new_label.csv"

new_label_df.to_csv(label_path, index=False)